In [4]:
class ConcatenatedCode:
    
    def __init__ (self, code1, code2):
        
        self.C1 = code1
        self.C2 = code2
        
        p, k = is_prime_power(self.C1.q, get_data = True)
        if self.C2.k != k:
            raise ValueError('Wrong pair of codes: Extension power of C1.q does not match C2.k')
        if self.C2.F != GF(p):
            raise ValueError('Wrong pair of codes: Extension field and base field do not fit together')
            
        print('Preferred message data type is: ', self.C1.message_type)
            
    def Encoding(self, m, out = 'pol'):
        
        data_type = self._DetermineInput(m)
        
        if data_type != self.C1.message_type:
            raise ValueError('Message data type does not match the preferred type. The preferred type is: ', self.C1.message_type)
            
        c = self.C1.Encoding(m, out = 'bin')
    
        c = self.C2.Encoding(c, out = out)
        
        return c
    
        
    def Decoding(self, r, out = 'bin'):
        
        c = self.C2.Decoding(r, out = 'bin')
        
        c = self.C1.Decoding(c, out = out)
        
        return c
    
    
    def _DetermineInput(self, data):

        # determine data type

        if isinstance(data, str) and all([(bit == '1' or bit == '0') for bit in data]):
            return 'bin'

        #elif isinstance(data, list):
        if data[0] == None:
            return 'none'
        elif data[0].parent() == self.C1.F:
            return 'pol'
        elif data[0].parent() == ZZ:
            return 'int'
        else:
            return 'unknown'

In [5]:
C = ConcatenatedCode(RSCode(n=15, k=7, q=2**4), RMCode(r=1, m=3))
#m = '10101010101011010'
m = [1,2,3,4,5,6]
c = C.Encoding(m, out = 'bin')
print(c)
d = C.Decoding(c, out = 'int')
print(d)

Preferred message data type is:  int
011010010110100111001100110000111010101010101010101010100011001101011010110000111001100110101010101010100000111110010110
[1, 2, 3, 4, 5, 6, 0]


In [1]:
import numpy as np
import itertools

class RMCode:
    
    def __init__(self, r, m):
            
        self.r = r
        self.m = m
        self.q = 2
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        self.message_type = 'bin'
        
        # Initializing field
        self.F = FiniteField(self.q)
        
        self.G = matrix(self.F, self.k, self.n, self.generateG(self.r, self.m))
        
        
    def generateG(self, r, m):
        ones = np.repeat(1, 2**m)
        bases = [np.array([(i // 2**a) % 2 for i in range(2**m)]) for a in range(m)]
        if r == 1:
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases)])
            return G
        elif r > 1:
            others = []
            for i in range(2,r+1):
                others.extend([math.prod(x) for x in itertools.combinations(bases, i)])
            
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases), np.stack(others)])
            return G
        else:
            raise ValueError('wrong value of r')
    
    
    def Encoding(self, message, zeropad = True, out = 'bin'):
        
        data_type = self._DetermineInput(message)
        
        if (data_type == 'pol' or data_type == 'bin'):
            message = list(message)
        else:
            raise ValueError('Wrong data type')
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')    
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        if out == 'pol':
            return vector(self.F, c)
        elif out == 'int':
            c = self._PolToInt(c)
            return c
        elif out == 'bin':
            c = self._PolToInt(c)
            c = self._IntToBitString(c)
            return c
        else:
            raise ValueError('Unrecognized output')
        
            
    def EncodeChunk(self, chunk):
        
        #chunk = list(chunk)
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received, out = 'bin'):
        
        #received = vector(self.F, received)
        data_type = self._DetermineInput(received)
        
        if data_type == 'pol':
            pass
        elif data_type == 'bin':
            received = vector(self.F, received)
        elif data_type == 'int':
            received = vector(self.F, received)
        else:
            raise ValueError('Wrong data type')
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        d = []
        
        for i in range(0,len(received),self.n):
            d.extend(self.DecodeChunk(received[i:i+self.n]))
            
        if out == 'pol':
            return vector(self.F, d)
        elif out == 'int':
            d = self._PolToInt(d)
            return d
        elif out == 'bin':
            d = self._PolToInt(d)
            d = self._IntToBitString(d)
            return d
        else:
            raise ValueError('Unrecognized output')
                
            
    def DecodeChunk(self, word):
        # Reed Decoding algorithm
        
        if (len(word) != self.n):
            raise ValueError('Invalid chunk size')
        
        decoded = []
        
        variables = [i for i in range(1, self.m + 1)]
        r = self.r
        G = self.G
        
        while (r > 0):
            current_variables = []
            current_variables.append(list(itertools.combinations(variables,r)))
            current_variables = list(current_variables)
            
            for i in range(binomial(self.m,r)-1, -1, -1):

                # compute the complementary set T
                T = [i for i in range(1,self.m+1)]
                for j in range(r):
                    T.remove(current_variables[0][i][j])
                    
                combinations_T = self.DecodingConvert(T)
                combinations_S = self.DecodingConvert(list(current_variables[0][i]))
                votings = []
                for i in range(len(combinations_T)):
                    voting = 0
                    for j in range(len(combinations_S)):
                        voting += word[combinations_T[i] + combinations_S[j]]

                    votings.append(voting)
                decoded = self.MajorityDecoding(votings) + decoded
                
            G_part = matrix(self.F, binomial(self.m,r), self.n, G[-binomial(self.m,r):][:])
            word_part = vector(self.F, decoded[:binomial(self.m,r)]) * G_part
            word += word_part
            
            G = G[:-binomial(self.m,r)][:]
            
            r -= 1
                
        # decode the first coefficient
        decoded = self.MajorityDecoding(word.list()) + decoded
        
        return vector(self.F, decoded)
    
    
    def DecodingConvert(self, elements): # takes as input one tuple
        combinations = []
        for i in range(len(elements)+1):
            combinations.append(list(itertools.combinations(elements,i)))
        combinations = list(combinations)
        integers = []
        for i in range(len(combinations)):
            for j in range(len(combinations[i])):
                l = [0 for i in range(self.m)]
                for k in range(len(combinations[i][j])):
                    l[combinations[i][j][k] - 1] = 1
                integers.append(ZZ(l,2))
        return integers

            
    def MajorityDecoding(self, word):
        # Decode by comparing the number of ones with the number of zeros
        
        if len(word) == 1:
            return word
        elif word.count(1) >= floor(len(word) / 2):
            return [1]
        elif word.count(1) < floor(len(word) / 2):
            return [0]
        else:
            return "decoding failure"
        
        
    def _DetermineInput(self, data):
        # determine data type

        if isinstance(data, str) and all([(bit == '1' or bit == '0') for bit in data]):

            return('bin')

        if data[0] == None:
            return('none')
        elif data[0].parent() == self.F:
            return('pol')
        elif data[0].parent() == ZZ:
            return('int')
        else:
            return('unknown')

    def _PolToInt(self, pol_array):

        # convert array of polynomial representation to array of integers

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.q))

        return(pol_out)
    
    def _IntToBitString(self, int_array):

        # Converts array of integers less than 2 to bit string

        if any([(item > (self.q - 1) or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return(''.join([format(item, '01b') for item in int_array]))

In [2]:
class RSCode:
    
    def __init__(self, n, k, q, alpha = None):
        
        if not (k < n and n <= q):
            raise ValueError('Invalid values for n, k, and q.')
            
        self.p0, self.m = is_prime_power(q, get_data = True)
            
        self.n = n
        self.k = k
        self.q = q
        
        self.d = self.n - self.k + 1
        
        self.tau = floor((self.n-self.k)/2)
        
        self.message_type = 'int'
        
        # Initializing field
        self.F = GF(self.q)
        self.R = PolynomialRing(self.F, 'X')
        self.p = self.F.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n)])
        else:
            self.alpha = alpha
        
        # Constructing generator matrix
        self.G = matrix(self.F, k, n, lambda i,j : self.alpha[j]**i)
        
    def Encoding(self, m, zeropad = True, out = 'bin'):
        
        # determine data type
        data_type = self._DetermineInput(m)
        
        if data_type == 'int':
            m = self._IntToPol(m)
        else:
            raise ValueError('Unrecognised input')
            
        
        rem = len(m) % self.k
        
        if rem != 0:
            if zeropad:
                m.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k):
            c.extend(self.EncodeChunk(m[i:i+self.k]))
        
        # Outputting decoded message in provided format
        if out == 'pol':
            return c
        elif out == 'bin':
            tmp = self._PolToInt(c)
            return self._IntToBitString(tmp)
        elif out == 'int':
            return(self._PolToInt(c))
        
        return c
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G
        return c
    
    def Decoding(self, r, out = 'pol'):
        
        # Determining input data type
        data_type = self._DetermineInput(r)

        if data_type == 'bin':
            r = self._BitStringToInt(r)
            r = self._IntToPol(r)
        elif data_type == 'int':
            r = self._IntToPol(r)
        elif data_type == 'pol':
            pass
        else:
            raise ValueError('Unrecognised input')
        
        # Check input size
        if len(r) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(r),self.n):
            c.extend(self.DecodeChunk(r[i:i+self.n]))
            
        
        # Outputting decoded message in provided format
        if out == 'pol':
            return c
        elif out == 'bin':
            tmp = self._PolToInt(c)
            return self._IntToBitString(tmp)
        elif out == 'int':
            return(self._PolToInt(c))
        
        return c
    
    def DecodeChunk(self, chunk):
        
        if len(chunk) != self.n:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.F, self.n, self.tau + self.k, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.F, self.n, self.tau + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau+self.k]))
        Q1 = self.R(list(sol[self.tau+self.k:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.F(0)]*(self.k-len(out)))

        return out
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q:
                raise ValueError('Invalid symbol')
            m_out.append(self.F(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)


    def _IntToBitString(self, int_array):

        # Converts array of integers less than 2 to bit string

        if any([(item > (self.q - 1) or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')
            
        number_of_bits = '0' + str(self.m) + 'b'

        return(''.join([format(item, number_of_bits) for item in int_array]))
    
    def _BitStringToInt(self, byte_string):

        #if(self.C1.q != 2**self.C2.k):
        #    raise ValueError('Invalid field size for byte representation')

        # Converts array of 8 bit binary representations to integers

        #if (len(byte_string) % self.k_RM != 0) or any([not (bit == '1' or bit == '0') for bit in byte_string]):
        #    raise ValueError('Invalid byte string')

        m_out = []
        for i in range(0,len(byte_string), self.m):
            current = byte_string[i:i+self.m]
            m_out.append(int("".join(str(x) for x in current), self.p0))
            
        return m_out
    
    
    
    def _DetermineInput(self, data):
        
        if isinstance(data, str) and all([(bit == '1' or bit == '0') for bit in data]):
            return 'bin'
    
        if isinstance(data, list):
        
            if data[0] == None:
                return 'none'
            elif data[0].parent() == self.F:
                return 'pol'
            elif data[0].parent() == ZZ:
                return 'int'
            else:
                return 'unknown'